In [1]:
import copy
import numpy as np
import itertools
import math

#ナンプレの問題をboardに保存
def data_input(n):
    global board_all,input_data
    input_data =[[]for _ in range(n)]
    data =list(input())
    board_all =[[[]for _ in range(9)] for _ in range(n)]
    new_line_error =0
    for number in range(n):
        row =0
        for i in range(81):
            if i %9==0 and i !=0:
                row +=1
            if data[i+number*81+new_line_error] =="0":
                board_all[number][row].append(" ")
                input_data[number].append("0")
            else:
                board_all[number][row].append(str(data[i+number*81+new_line_error]))
                input_data[number].append(str(data[i+number*81+new_line_error]))
        new_line_error +=1
        input_data[number] ="".join(input_data[number])
    return board_all,input_data
def ans_candidate_set():
    global ans_candidate
    pre =set(["1","2","3","4","5","6","7","8","9"])
    ans_candidate =[[copy.copy(pre) for _ in range(9)] for _ in range(9)]#答えの候補
    return ans_candidate
    


#全て数字が埋まっているかの判定
def finish(board):
    for row in range(9):
        for colum in range(9):
            if board[row][colum]==" ":
                return False
    return True

#残り物の常識
def level1(ans_candidate,board):
    lv1_flag =False
    for row in range(9):
        for colum in range(9):
            if board[row][colum]==" ":
                if len(list(ans_candidate[row][colum]))==1:
                    lv1_flag =True
                    board[row][colum]  =ans_candidate[row][colum].pop()
                    ans_candidate[row][colum] =set([])
            else:
                ans_candidate[row][colum] =set([])
              
    return lv1_flag,ans_candidate,board
            

#行の常識               
def level2_row(ans_candidate,board):
    flag_lv2_row =False
    #各行の数字を取得
    for row in range(9):
        board_alredy_in =set([])
        for colum in range(9):
            if board[row][colum]!=" ":
                board_alredy_in.add(board[row][colum])

        #行の常識を使い、候補を絞り込む
        for i in range(9):
            if board[row][i] ==" ":
                if ans_candidate[row][i] - board_alredy_in !=ans_candidate[row][i]:
                    flag_lv2_row =True
                    ans_candidate[row][i] =ans_candidate[row][i] - board_alredy_in
                    #print("row",row,colum,ans_candidate[row][colum])
    return flag_lv2_row,ans_candidate,board
               
#列の常識               
def level2_colum(ans_candidate,board):
    flag_lv2_colum =False
    #各列の数字を取得
    for colum in range(9):
        board_alredy_in =set()
        for row in range(9):
            if board[row][colum]!=" ":
                board_alredy_in.add(board[row][colum])
        #列の常識を使い、候補を絞り込む
        for i in range(9):
            if board[i][colum] ==" ":
                if ans_candidate[i][colum] - board_alredy_in !=ans_candidate[i][colum]:
                    flag_lv2_colum =True
                    #print("colum",row,colum,ans_candidate[row][colum])
                    ans_candidate[i][colum] = ans_candidate[i][colum] - board_alredy_in
    return flag_lv2_colum,ans_candidate,board

#ブロックの常識               
def level2_block(ans_candidate,board):
    flag_lv2_block =False
    #各ブロックの数字を取得
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        board_alredy_in =set()
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum]!=" ":
                    board_alredy_in.add(board[row+block_row][colum+block_colum])
        
        #ブロックの常識を使い、候補を絞り込む
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum] ==" ":
                    if ans_candidate[row+block_row][colum+block_colum] - board_alredy_in !=ans_candidate[row+block_row][colum+block_colum]:
                        flag_lv2_block =True
                       # print("block",row+block_row,colum+block_colum,ans_candidate[row+block_row][colum+block_colum])
                        ans_candidate[row+block_row][colum+block_colum] =ans_candidate[row+block_row][colum+block_colum] - board_alredy_in
    return flag_lv2_block,ans_candidate,board
    
#一ヶ所確定   
def level3_one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number): 
    row_alredy_in =set() #着目している数字がある行
    colum_alredy_in =set() #着目している数字がある列
    block_alredy_in =set() #着目している数字があるブロック
    block_number =0 #ブロックに番号を振る
        
    #着目している数字の探索、入らない場所の特定
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        for row in range(3):
            for colum in range(3):
                if board[row+block_row][colum+block_colum] ==" ":continue
                if int(board[row+block_row][colum+block_colum]) ==target_number:
                    row_alredy_in.add(row+block_row)
                    colum_alredy_in.add(colum+block_colum)
                    block_alredy_in.add(block_number)
        block_number +=1
    return row_alredy_in,colum_alredy_in,block_alredy_in






def level3_one_place_confirmed_block(ans_candidate,board):#着目している数字(target_number)が、あるブロックにおいて1度のみしか現れないかどうか判定する  
    flag_lv3_one_place_confirmed_block =False
    for target_number in range(1,10):
        row_alredy_in,colum_alredy_in,block_alredy_in =level3_one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number)#(target_number:着目する数字) target_numberが入らない座標
        can_fill_in_block =[] #ブロックにおいて着目した場合に数字を埋めることができる座標
        block_number =0
        for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
            block_row =block[0]
            block_colum =block[1]
            if block_number in block_alredy_in:
                block_number +=1
                continue
            #ブロックにおいて着目した場合に数字を埋めることができる座標が１つの時にboardを更新する・        
            if len(can_fill_in_block) ==1:
                board[can_fill_in_block[0][0]][can_fill_in_block[0][1]] =str(target_number)
                flag_lv3_one_place_confirmed_block =True
            can_fill_in_block =[]
            for row in range(3):
                for colum in range(3):
                    if row+block_row in row_alredy_in:
                       break
                    if board[row+block_row][colum+block_colum] !=" " or  colum+block_colum in colum_alredy_in:continue
                    #着目している数字があるブロックにおいて1度のみ入れることが可能かどうか判定する    
                    if len(can_fill_in_block) ==0:
                        can_fill_in_block.append([row+block_row,colum+block_colum])
                    else:
                        can_fill_in_block.append("damy")
                        break
            block_number +=1
    return flag_lv3_one_place_confirmed_block,ans_candidate,board
               
               
def level3_one_place_confirmed_row(ans_candidate,board):#着目している数字(target_number)が、ある行において1度のみしか現れないかどうか判定する  
    flag_lv3_one_place_confirmed_row =False
    for target_number in range(1,10):
        row_alredy_in,colum_alredy_in,block_alredy_in =level3_one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number)#target_number:着目する数字
        can_fill_in_row =[] #行において着目した場合に数字を埋めることができる座標
        for row in range(9):
            if len(can_fill_in_row) ==1:#行において着目した場合に数字を埋めることができる座標が１つの時にboardを更新する・  
                board[can_fill_in_row[0][0]][can_fill_in_row[0][1]] =str(target_number)
                flag_lv3_one_place_confirmed_row =True
            can_fill_in_row =[]
            for colum in range(9):
                if row in row_alredy_in:
                    break
                if board[row][colum] !=" " or  colum in colum_alredy_in:continue
                #着目している数字がある行において1度のみ入れることが可能かどうか判定する  
                if len(can_fill_in_row) ==0:
                    can_fill_in_row.append([row,colum])
                else:
                    can_fill_in_row.append("damy")
                    break
    return flag_lv3_one_place_confirmed_row,ans_candidate,board
                
        
        

def level3_one_place_confirmed_colum(ans_candidate,board):#着目している数字(target_number)が、ある列において1度のみしか現れないかどうか判定する  
    flag_lv3_one_place_confirmed_colum =False
    for target_number in range(1,10):
        row_alredy_in,colum_alredy_in,block_alredy_in =level3_one_place_confirmed_prohibit_row_colum_block(ans_candidate,board,target_number)#target_number:着目する数字
        can_fill_in_colum =[] #列において着目した場合に数字を埋めることができる座標
        for colum in range(9):
            if len(can_fill_in_colum) ==1:#列において着目した場合に数字を埋めることができる座標が１つの時にboardを更新する・ 
                board[can_fill_in_colum[0][0]][can_fill_in_colum[0][1]] =str(target_number)
                flag_lv3_one_place_confirmed_colum =True
            can_fill_in_colum =[]
            if colum in colum_alredy_in:continue
            for row in range(9):
                if board[row][colum] !=" " or  row in row_alredy_in:continue
                if len(can_fill_in_colum) ==0:#着目している数字がある列において1度のみ入れることが可能かどうか判定する  
                    can_fill_in_colum.append([row,colum])
                else:
                    can_fill_in_colum.append("damy")
                    break
    return flag_lv3_one_place_confirmed_colum,ans_candidate,board
       
       
#二国同盟/定員確定法(ブロックに着目)
def level4_two_country_block(ans_candidate,board):
    flag_lv4_two_country_block =False
    for block in [[0,0],[0,3],[0,6],[3,0],[3,3],[3,6],[6,0],[6,3],[6,6]]:
        block_row =block[0]
        block_colum =block[1]
        
        two_ans_candidate =[] #答えの候補が２つのもの
        del_ans_candidate =[] #ブロック内の候補から削除できる数字
        
        two_ans_candidate_point =[] #答えの候補が２つのものの座標
        del_ans_candidate_point =[] #削除できる候補の数字がある座標(削除しない座標)
        for row in range(3):
            for colum in range(3):
                if len(list(ans_candidate[row+block_row][colum+block_colum])) ==2 and board[row+block_row][colum+block_colum] ==" ":#答えの候補が２つのものを判断する
                    two_ans_candidate.append(copy.copy(ans_candidate[row+block_row][colum+block_colum]))
                    two_ans_candidate_point.append([row+block_row,colum+block_colum])
                    for i in range(len(two_ans_candidate)-1):#答えの候補が２つであり同一のものを探索する
                        if two_ans_candidate[i] ==ans_candidate[row+block_row][colum+block_colum]:
                            del_ans_candidate.append(two_ans_candidate[i]) #削除できる候補をdel_ans_candidateに保存する
                            del two_ans_candidate[i]
                            del two_ans_candidate[-1]
                            
                            del_ans_candidate_point.append([two_ans_candidate_point[i],two_ans_candidate_point[-1]])
                            del two_ans_candidate_point[i]
                            del two_ans_candidate_point[-1]
                            break
                        
                            
        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            for j in range(2):
                del_number =str(del_ans_candidate[i].pop()) #削除する数字
                for row in range(3):
                    for colum in range(3):
                        if (del_ans_candidate_point[i][0][0] ==row+block_row and del_ans_candidate_point[i][0][1] ==colum+block_colum) or (del_ans_candidate_point[i][1][0] ==row+block_row and del_ans_candidate_point[i][1][1] ==colum+block_colum):continue
                        if (del_number in ans_candidate[row+block_row][colum+block_colum]):
                            ans_candidate[row+block_row][colum+block_colum].remove(del_number)
                            flag_lv4_two_country_block =True
            
    return flag_lv4_two_country_block,ans_candidate,board
    
#二国同盟/定員確定法(行に着目)
def level4_two_country_row(ans_candidate,board):
    flag_lv4_two_country_row =False
    for row in range(9):
        two_ans_candidate =[] #答えの候補が２つのもの
        del_ans_candidate =[] #探査中の行の候補から削除できる数字
        
        two_ans_candidate_point =[] #答えの候補が２つのものの座標
        del_ans_candidate_point =[] #削除できる候補の数字がある座標(削除しない座標)
        for colum in range(9):
            if len(list(ans_candidate[row][colum])) ==2 and board[row][colum] ==" ":#答えの候補が２つのものを判断する
                    two_ans_candidate.append(copy.copy(ans_candidate[row][colum]))
                    two_ans_candidate_point.append([row,colum])
                    for i in range(len(two_ans_candidate)-1):#答えの候補が２つであり同一のものを探索する
                        if two_ans_candidate[i] ==ans_candidate[row][colum]:
                            del_ans_candidate.append(two_ans_candidate[i]) #削除できる候補をdel_ans_candidateに保存する
                            del two_ans_candidate[i]
                            del two_ans_candidate[-1]
                            
                            del_ans_candidate_point.append([two_ans_candidate_point[i],two_ans_candidate_point[-1]])
                            del two_ans_candidate_point[i]
                            del two_ans_candidate_point[-1]
                            break
               
        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            for j in range(2):
                del_number =str(del_ans_candidate[i].pop()) #削除する数字
                for colum in range(9):
                    if (del_ans_candidate_point[i][0][0] ==row and del_ans_candidate_point[i][0][1] ==colum) or (del_ans_candidate_point[i][1][0] ==row and del_ans_candidate_point[i][1][1] ==colum):continue
                    if (del_number in ans_candidate[row][colum]):
                        ans_candidate[row][colum].remove(del_number)
                        flag_lv4_two_country_row =True
    return flag_lv4_two_country_row,ans_candidate,board
    
#二国同盟/定員確定法(列に着目)
def level4_two_country_colum(ans_candidate,board):
    flag_lv4_two_country_colum =False
    for colum in range(9):
        two_ans_candidate =[] #答えの候補が２つのもの
        del_ans_candidate =[] #探査中の行の候補から削除できる数字
        
        two_ans_candidate_point =[] #答えの候補が２つのものの座標
        del_ans_candidate_point =[] #削除できる候補の数字がある座標(削除しない座標)
        for row in range(9):
            if len(list(ans_candidate[row][colum])) ==2 and board[row][colum] ==" ":#答えの候補が２つのものを判断する
                    two_ans_candidate.append(copy.copy(ans_candidate[row][colum]))
                    two_ans_candidate_point.append([row,colum])
                    for i in range(len(two_ans_candidate)-1):#答えの候補が２つであり同一のものを探索する
                        if two_ans_candidate[i] ==ans_candidate[row][colum]:
                            del_ans_candidate.append(two_ans_candidate[i]) #削除できる候補をdel_ans_candidateに保存する
                            del two_ans_candidate[i]
                            del two_ans_candidate[-1]
                            
                            del_ans_candidate_point.append([two_ans_candidate_point[i],two_ans_candidate_point[-1]])
                            del two_ans_candidate_point[i]
                            del two_ans_candidate_point[-1]
                            break
               
        #削除できる候補をans_candidateから削除する
        for i in range(len(del_ans_candidate)):
            for j in range(2):
                del_number =str(del_ans_candidate[i].pop()) #削除する数字
                for row in range(9):
                    if (del_ans_candidate_point[i][0][0] ==row and del_ans_candidate_point[i][0][1] ==colum) or (del_ans_candidate_point[i][1][0] ==row and del_ans_candidate_point[i][1][1] ==colum):continue
                    if (del_number in ans_candidate[row][colum]):
                        ans_candidate[row][colum].remove(del_number)
                        flag_lv4_two_country_colum =True
    return flag_lv4_two_country_colum,ans_candidate,board
 


#四角対角線(行に着目)
def level6_square_diagonal_row(ans_candidate,board):
    flag_lv6_square_diagonal_row =False
    for target_number in range(1,10):#target_numberは着目する数字
        two_ans_candidate_target_number =[]#target_numberが入る候補が２つの座標を保存する
        for row in range(9):
            row_target_number_cnt =0#ある行に関してtraget_numberが入る可能性がある箇所の個数を記録する
            target_number_in_point =[row] #ある行に関してtraget_numberが入る可能性がある箇所の[行の座標]「列の座標」を保存する→[行の座標,列の座標（1つ目）,列の座標（2つ目）]
            for colum in range(9): #ある行に関してtraget_numberが入る可能性がある箇所の個数を求める
                if str(target_number) in ans_candidate[row][colum]:
                    row_target_number_cnt +=1
                    target_number_in_point.append(colum)
                    if row_target_number_cnt ==4: #traget_numberが入る可能性がある箇所が３つ以上であれば四角対角線は使えない
                        break
            if row_target_number_cnt ==2:#ある行に関してtraget_numberが入る可能性がある箇所の個数が２つの時、四角対角線を使える可能性がある
                two_ans_candidate_target_number.append(target_number_in_point)
       
        
        for search in  itertools.combinations(two_ans_candidate_target_number, 2):#候補のそれぞれの列が２つとも一致する行の組み合わせを探す。

            del_col =[]#target_numberを候補から消せる列
            if search[0][1] ==search[1][1] and search[0][2] ==search[1][2]:
                del_col.append(search[0][1])
                del_col.append(search[0][2])
                for colum in del_col:
                    for row in range(9):
                        if (str(target_number) in ans_candidate[row][colum]):
                            if search[0][0] ==row or search[1][0] ==row:continue
                            ans_candidate[row][colum].remove(str(target_number))
                            flag_lv6_square_diagonal_row =True
    return flag_lv6_square_diagonal_row,ans_candidate,board 

#四角対角線(列に着目)
def level6_square_diagonal_colum(ans_candidate,board):
    flag_lv6_square_diagonal_colum =False
    for target_number in range(1,10):#target_numberは着目する数字
        two_ans_candidate_target_number =[]#target_numberが入る候補が２つの座標を保存する
        for colum in range(9):
            row_target_number_cnt =0#ある列に関してtraget_numberが入る可能性がある箇所の個数を記録する
            target_number_in_point =[colum] #ある列に関してtraget_numberが入る可能性がある箇所の[行の座標]「列の座標」を保存する→[列の座標,行の座標（1つ目）,行の座標（2つ目）]
            for row in range(9): #ある行に関してtraget_numberが入る可能性がある箇所の個数を求める
                if str(target_number) in ans_candidate[row][colum]:
                    row_target_number_cnt +=1
                    target_number_in_point.append(row)
                    if row_target_number_cnt ==4: #traget_numberが入る可能性がある箇所が３つ以上であれば四角対角線は使えない
                        break
            if row_target_number_cnt ==2:#ある列に関してtraget_numberが入る可能性がある箇所の個数が２つの時、四角対角線を使える可能性がある
                two_ans_candidate_target_number.append(target_number_in_point)
       
        for search in  itertools.combinations(two_ans_candidate_target_number, 2):#候補のそれぞれの列が２つとも一致する行の組み合わせを探す。
            del_row =[]#target_numberを候補から消せる行
            if search[0][1] ==search[1][1] and search[0][2] ==search[1][2]:
                del_row.append(search[0][1])
                del_row.append(search[0][2])
                for row in del_row:
                    for colum in range(9):
                        if (str(target_number) in ans_candidate[row][colum]):
                            if search[0][0] ==colum or search[1][0] ==colum:continue
                            ans_candidate[row][colum].remove(str(target_number))
                            flag_lv6_square_diagonal_colum =True

    return flag_lv6_square_diagonal_colum,ans_candidate,board 



                            
    
    
    
    
    
    
    
    
    
                       
#最終手段（全探査）

def fi(s): #次にどこのマスに数字を入れればいいか(入る候補が最も少ないのはどこか)を判定します
    _ = []
    used_yoko = [set([s[k%9+l*9] for l in range(9)]) for k in range(9)] #あらかじめ、各列各行各ブロックで使用済みの数字をリストアップしておきます
    used_tate = [set([s[k*9+l] for l in range(9)]) for k in range(9)]
    used_block = [set([s[k//3*27+k%3*3+l//3*9+l%3] for l in range(9)]) for k in range(9)]
    for k in range(81):
      if s[k] != "0":
        _ += [0]
      else:
        ss = used_yoko[k%9]|used_tate[k//9]|used_block[k//27*3+k%9//3]
        if "0" in ss:
          ss.remove("0")
        _ += [len(set(ss))]
    k_next = _.index(max(_))
    ss_next =used_yoko[k_next%9]|used_tate[k_next//9]|used_block[k_next//27*3+k_next%9//3]
    return k_next,set(map(str,range(1,10)))-set(ss_next)

def dfs(i,t): #全探索を行う部分です
  global board_last_means
  if i == sudoku.count("0"):
    board_last_means =np.array(list(t)).reshape(9,9)
  else:
    k,next_number = fi(t)
    for j in next_number:
      dfs(i+1,t[:k]+j+t[k+1:])
    
def last_means(data):#全探査をしたか判定
    global sudoku
    sudoku = data 
    dfs(0,sudoku)
    return True

def execution(ans_candidate,board,level):#それぞれの解法を実行
    while True:
        flag,ans_candidate,board =level1(ans_candidate,board)
        if finish(board):
            break
        flag2_row,ans_candidate,board =level2_row(ans_candidate,board)
        flag2_colum,ans_candidate,board =level2_colum(ans_candidate,board)
        flag2_block,ans_candidate,board =level2_block(ans_candidate,board)
        if  flag2_row or flag2_colum  or flag2_block:
            level =max(level,2)
            flag,ans_candidate,board =level1(ans_candidate,board)
            continue
        flag3_row,ans_candidate,board =level3_one_place_confirmed_row(ans_candidate,board)
        flag3_colum,ans_candidate,board =level3_one_place_confirmed_colum(ans_candidate,board)
        flag3_block,ans_candidate,board =level3_one_place_confirmed_block(ans_candidate,board)
        if  flag3_row or flag3_colum  or flag3_block:
            level =max(level,3)
            flag,ans_candidate,board =level1(ans_candidate,board)
            continue 
        flag4_row,ans_candidate,board =level4_two_country_row(ans_candidate,board)
        flag4_colum,ans_candidate,board =level4_two_country_colum(ans_candidate,board)
        flag4_block,ans_candidate,board =level4_two_country_block(ans_candidate,board)
        if  flag4_row or flag4_colum  or flag4_block:
            level =max(level,4)
            flag,ans_candidate,board =level1(ans_candidate,board)
            continue
        
        flag6_row,ans_candidate,board =level6_square_diagonal_row(ans_candidate,board)
        flag6_colum,ans_candidate,board =level6_square_diagonal_colum(ans_candidate,board)
        if flag6_row or flag6_colum:
            level =max(level,6)
            flag,ans_candidate,board =level1(ans_candidate,board)
            continue
            
        break
    return level

  
def output(level,input_data):
    for i in range(81): #入力データが正常かどうか判断
        if input_data[i] ==" ":
            level ="error"

            return level
    for i in range(9):
        if "".join(board[i])!="".join(board_last_means[i]):
            level =0
            return level
    return level
    



In [4]:
n =int(input())
data_input(n)
level_array =[0 for _ in range(8)]




for number in range(n):
    board =board_all[number]
    ans_candidate =ans_candidate_set()
    level =1
    level =execution(ans_candidate,board,level)
    last_means(input_data[number])
    level = output(level,input_data[number])
    if level =="error":
        print(number,"error")
        break
    else:
        level_array[level] +=1
    print(number,":",level)
print(level_array)
    
print(board)
for i in range(9):
    print("".join(board[i]))


1
730002080680470020019508647060900004342680000190050830000720403906005010003891500
0 : 2
[0, 0, 1, 0, 0, 0, 0, 0]
[['7', '3', '4', '1', '6', '2', '9', '8', '5'], ['6', '8', '5', '4', '7', '9', '3', '2', '1'], ['2', '1', '9', '5', '3', '8', '6', '4', '7'], ['5', '6', '8', '9', '1', '3', '2', '7', '4'], ['3', '4', '2', '6', '8', '7', '1', '5', '9'], ['1', '9', '7', '2', '5', '4', '8', '3', '6'], ['8', '5', '1', '7', '2', '6', '4', '9', '3'], ['9', '2', '6', '3', '4', '5', '7', '1', '8'], ['4', '7', '3', '8', '9', '1', '5', '6', '2']]
734162985
685479321
219538647
568913274
342687159
197254836
851726493
926345718
473891562


In [87]:
print(level_array)

[3000, 0, 0, 0, 0, 0, 0, 0]


In [10]:
#レベル２
[0, 0, 2996, 4, 0, 0, 0, 0]
#レベル３
[89, 0, 93, 2619, 199, 0, 0, 0]
#レベル４
[424, 0, 0, 1761, 815, 0, 0, 0]
#レベル５
[2494, 0, 0, 25, 481, 0, 0, 0]
#レベル７
[3000, 0, 0, 0, 0, 0, 0, 0]

[3000, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
#レベル５
[2356, 0, 0, 25, 481, 0, 138, 0]